In [1]:
import numpy as np
import pandas as pd

import keras
from keras import models
from keras import layers
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, average_precision_score


In [2]:
# accuracy, AUC and F-1
# f1_score(), accuracy_score(), auc(), average_precision_score()

def evaluate (y_true_train, y_pred_prob_train, y_true_test, y_pred_prob_test,  threshold=0.5):
    y_pred_train = pd.Series(np.reshape(y_pred_prob_train, (1,np.product(y_pred_prob_train.shape)))[0])
    y_pred_test = pd.Series(np.reshape(y_pred_prob_test, (1,np.product(y_pred_prob_test.shape)))[0])
    
    y_pred_train = y_pred_train.apply(lambda x: 1 if x >= threshold else 0 )
    y_pred_test = y_pred_test.apply(lambda x: 1 if x >= threshold else 0 )
    
    confusion_matrix = pd.crosstab(y_true_test, y_pred_test, rownames=['Actual'], colnames=['Predicted'])
    print('Test Set Confusion Matrix')
    print(confusion_matrix)
    
    return pd.DataFrame.from_records([[f1_score(y_true_train, y_pred_train), 
                                       accuracy_score(y_true_train, y_pred_train), 
                                       roc_auc_score(y_true_train, y_pred_prob_train), 
                                       average_precision_score(y_true_train, y_pred_prob_train),
                                      (y_pred_train.sum() / y_pred_train.count())], 
                                      
                                      [f1_score(y_true_test, y_pred_test), 
                                       accuracy_score(y_true_test, y_pred_test), 
                                       roc_auc_score(y_true_test, y_pred_prob_test), 
                                       average_precision_score(y_true_test, y_pred_prob_test),
                                      (y_pred_test.sum() / y_pred_test.count())]], 
                                     index=['Train', 'Test'], 
                                     columns=['f1_score', 'accuracy_score', 'auc', 'average_precision_score', 'positive_ratio'])

In [3]:
data = pd.read_csv('Call Details-Data.csv')

def prepare_data(data=data, scaler = StandardScaler()):
    data.drop('Phone Number', axis=1, inplace=True)

    data['Churn'] = data['Churn'].apply(lambda x: 1 if x else 0)

    x = data.drop('Churn', axis=1)
    y = data['Churn']
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=0)
    
    # Scale Train
    if scaler is not None:
        scaler_train = scaler
        scaler_train.fit(x_train)
        x_train_s = scaler_train.transform(x_train)
        x_train = pd.DataFrame(x_train_s, index=x_train.index, columns=x_train.columns)

        # Scale Test
        scaler_test = scaler
        scaler_test.fit(x_test)
        x_test_s = scaler_test.transform(x_test)
        x_test = pd.DataFrame(x_test_s, index=x_test.index, columns=x_test.columns)
    
    return x_train, x_test, y_train, y_test
    

In [4]:
x_train, x_test, y_train, y_test = prepare_data(data=data, scaler=StandardScaler())

In [5]:
print('Positive Ratio', data.Churn.sum() / data.Churn.count())

Positive Ratio 0.10409789076244885


In [6]:
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(x_train.shape[1],)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

history = model.fit(x_train, y_train, epochs=5, batch_size=10,  verbose=1, validation_split=0.2)

test_acc_score, test_f1_score = model.evaluate(x_test, y_test)

y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)
model.summary()

Epoch 1/5
5423/5423 [==============================] - 21s 4ms/step - loss: 0.3349 - acc: 0.8945 - val_loss: 0.3244 - val_acc: 0.8961
Epoch 2/5
5423/5423 [==============================] - 18s 3ms/step - loss: 0.3207 - acc: 0.8953 - val_loss: 0.3152 - val_acc: 0.8960
Epoch 3/5
5423/5423 [==============================] - 18s 3ms/step - loss: 0.3155 - acc: 0.8954 - val_loss: 0.3137 - val_acc: 0.8964
Epoch 4/5
5423/5423 [==============================] - 10s 2ms/step - loss: 0.3110 - acc: 0.8958 - val_loss: 0.3208 - val_acc: 0.8964
Epoch 5/5
1044/1044 [==============================] - 1s 1ms/step - loss: 0.3092 - acc: 0.8967
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                1024      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
__________________

In [10]:
evaluate(y_train, y_train_pred, y_test, y_test_pred, threshold=0.2)

Test Set Confusion Matrix
Predicted     0    1
Actual              
0          8593  815
1          1430  144


,f1_score,accuracy_score,auc,average_precision_score,positive_ratio
Train,0.279827,0.862331,0.720394,0.261736,0.08695
Test,0.247969,0.853031,0.698533,0.224851,0.09156


In [8]:
y_test_pred

array([[0.13729462],
       [0.05941761],
       [0.1063467 ],
       ...,
       [0.09618428],
       [0.04914737],
       [0.0828504 ]], dtype=float32)